In [6]:
import cv2
import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
import os
from deepface import DeepFace as df
#from retinaface import RetinaFace as m
#from PIL import Image
#import imagehash
import pandas as pd
# from pymongo import MongoClient
# from deepface.commons import functions
# from tqdm import tqdm
import mediapipe as mp

In [7]:
%%time
video = cv2.VideoCapture('Data/Video/input2.MOV')
count = 0
frames_arr = []
while video.isOpened():
    ret, frame = video.read()

    if not ret:
        break
    frames_arr.append(frame)                                         # Frames stored in frames_arr
    count += 1
video.release()
print(len(frames_arr), count)


img = frames_arr[0]
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)

0 0


IndexError: list index out of range

In [ ]:
%%time
frames = []
frames.append(frames_arr[0])
orb = cv2.ORB_create()
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
for i in range(1, len(frames_arr)):
    keypoints1, descriptors1 = orb.detectAndCompute(frames[-1], None)
    keypoints2, descriptors2 = orb.detectAndCompute(frames_arr[i], None)
    matches = bf.match(descriptors1, descriptors2)
    
    #matches = sorted(matches, key=lambda x: x.distance)
    
    if len(matches) < 200:
        frames.append(frames_arr[i])
print(len(frames))

In [ ]:
output_folder='Data/intermediate_results/original_frames'
for i, frame in enumerate(frames):
        cv2.imwrite(os.path.join(output_folder, f'frame_{i}.jpg'), frame)

In [ ]:
def calculate_blurriness(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    return cv2.Laplacian(gray, cv2.CV_64F).var()

def remove_blurred_frames(frame_list, threshold=60):
    sharp_frames = []
    count=0
    for frame in frame_list:
        sharpness = calculate_blurriness(frame)
        print("frame",count," ",sharpness)
        count+=1
        if sharpness > threshold:
            sharp_frames.append(frame)
    return sharp_frames


# Assuming frames is a list of numpy arrays where each array represents a frame in BGR format
# Adjust the threshold value as per your requirement
sharp_frames = remove_blurred_frames(frames, threshold=60)

# Now sharp_frames will contain the list of frames without blurred ones

In [ ]:
output_folder='Data/intermediate_results/sharp_frames'
for i, frame in enumerate(sharp_frames):
        cv2.imwrite(os.path.join(output_folder, f'frame_{i}.jpg'), frame)

In [ ]:
%%time
detected_faces = []
img = frames_arr[0]
mp_face_detction = mp.solutions.face_detection
for frame in frames:
    with mp_face_detction.FaceDetection(model_selection = 1, min_detection_confidence = 0.5) as face_detection:
        results = face_detection.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        for detection in results.detections:
            box = detection.location_data.relative_bounding_box
            x_start, y_start = int(box.xmin * img.shape[1]), int(box.ymin * img.shape[0])
            x_end, y_end = int((box.xmin + box.width) * img.shape[1]), int((box.ymin + box.height) * img.shape[0])

            if x_start < 0 or y_start < 0:
                continue

            face = frame[y_start:y_end, x_start:x_end]
            face = cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
            detected_faces.append(face)
print(len(detected_faces))

In [ ]:
output_folder='Data/intermediate_results/detected_faces'
for i, face in enumerate(detected_faces):
        face=cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        cv2.imwrite(os.path.join(output_folder, f'face_{i}.jpg'), face)

In [ ]:
faces = []
faces.append(cv2.convertScaleAbs(detected_faces[0]))
orb = cv2.ORB_create()
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
for i in range(1, len(detected_faces)):
    img2 = cv2.convertScaleAbs(detected_faces[i])

    

    keypoints1, descriptors1 = orb.detectAndCompute(faces[-1], None)
    keypoints2, descriptors2 = orb.detectAndCompute(img2, None)

    if descriptors2 is None:
        
        continue

    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    matches = bf.match(descriptors1, descriptors2)

    #matches = sorted(matches, key=lambda x: x.distance)

    if len(matches) < 200:
        faces.append(detected_faces[i])

detected_faces[:]=faces[:]
print(len(detected_faces))

plt.imshow(detected_faces[2])

In [ ]:
output_folder='Data/intermediate_results/detected_faces_similar_removed'
for i, face in enumerate(detected_faces):
        face=cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        cv2.imwrite(os.path.join(output_folder, f'face_{i}.jpg'), face)

In [ ]:


def calculate_blurriness(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    return cv2.Laplacian(gray, cv2.CV_64F).var()

def remove_blurred_faces(face_list, threshold=100):
    sharp_faces = []
    for face_image in face_list:
        sharpness = calculate_blurriness(face_image)
        print(sharpness)
        if sharpness > threshold:
            sharp_faces.append(face_image)
    return sharp_faces

# Example usage:

# Assuming detected_faces is a list of numpy arrays where each array represents a face image in BGR format
# Adjust the threshold value as per your requirement
sharp_faces = remove_blurred_faces(detected_faces, threshold=100)

# Now sharp_faces will contain the list of faces without blurred or unclear ones

In [ ]:
output_folder='Data/intermediate_results/sharp_faces'
for i, face in enumerate(detected_faces):
        face=cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        cv2.imwrite(os.path.join(output_folder, f'face_{i}.jpg'), face)

In [ ]:
models=[]
res=[]
present=[]
for i in range(len(detected_faces)):
    model=df.find(img_path=detected_faces[i],db_path='Data/known_faces',model_name="Facenet",distance_metric="euclidean",enforce_detection=False,normalization="Facenet",detector_backend='mediapipe')
    models.append(model)
print()
count = 0
for model in models:
    if len(model[0]) > 0:
        name=model[0]['identity'].values[0].split('\\')[-1].split('/')[-1].split('.')[-2]
        print(count , "_ ", name)
        if name not in present:
            present.append(name)
    else:
        print('Unknown Face detected')
    count += 1